In [41]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, FullScreenControl, DrawControl, Marker, \
                        Circle, LayersControl, Polyline, Rectangle, WidgetControl, LayerGroup, AwesomeIcon, Icon, ScaleControl, GeoJSON
from ipywidgets import IntSlider, VBox, HBox, Tab, Button, Dropdown, Text, Layout, IntRangeSlider, Checkbox, Label, RadioButtons, HTML, FloatText
import math
import aerocalc3.unit_conversion as convert
import re

import pyproj

geo = pyproj.Geod(ellps="WGS84")
constant_g = 9.80665

user_lat = 15.0
user_long = -15.0

In [42]:

m = Map(
    basemap=basemaps.OpenTopoMap,
    center=(46.5, 6.5),
    scroll_wheel_zoom=True,
    zoom=5
)

m.add_control(FullScreenControl())

ee_basemaps = {}

# Loops through all ipyleaflet basemaps

for item in basemaps.values():
#     print(item.get("name", "No name."))
#     print(item.get("url", "No url."))
#     print(item.keys())
    try:
        name = item["name"]
        basemap = "basemaps.{}".format(name)
        ee_basemaps[name] = basemap_to_tiles(eval(basemap))
    except:
        try:
            for sub_item in item:
                name = item[sub_item]["name"]
                basemap = "basemaps.{}".format(name)
                basemap = basemap.replace("Mids", "Modis")
                ee_basemaps[name] = basemap_to_tiles(eval(basemap))
        except:
            pass

# Adds a Dropdown widget
dropdown = Dropdown(
    options=list(ee_basemaps.keys()),
    value="OpenTopoMap",
    description="Basemaps",
)

# Handles Dropdown control event
def on_click(change):
    basemap_name = change["new"]
    old_basemap = m.layers[-1]
    m.substitute_layer(old_basemap, ee_basemaps[basemap_name])

dropdown.observe(on_click, "value")

# Adds control to the map
basemap_control = WidgetControl(widget=dropdown, position="topright")
m.add_control(basemap_control)

# control = LayersControl(position='topleft')
# m.add_control(control)

# maison = Marker(name='Maison', location=(43.559837, 1.314936))
# m.add_layer(maison)

# draw_control = DrawControl()
# draw_control.polyline =  {
#     "shapeOptions": {
#         "color": "purple",
#         "weight": 10,
#         "opacity": 0.2
#     }
# }
# draw_control.polygon = {}
# draw_control.circlemarker = {}
# draw_control.rectangle = {}

# m.add_control(draw_control)
m.add_control(ScaleControl(position='bottomleft', imperial = False, max_width = 150))



In [43]:
def _lat_label_format(option):

    if option == "Deg.dec":
        return "+/-DD.ddd"
    elif option == "DegMin.dec":
        return "N/SDDMM.ddd"
            
    elif option == "DegMinSec.dec":
        return "N/SDDMMSS.ddd"
    else:
        return "Not defined"

def _lon_label_format(option):

    if option == "Deg.dec":
        return "+/-DDD.ddd"
    elif option == "DegMin.dec":
        return "W/EDDDMM.ddd"
            
    elif option == "DegMinSec.dec":
        return "W/EDDDMMSS.ddd"
    else:
        return "Not defined"

geo_choice = RadioButtons(options = ["Deg.dec","DegMin.dec","DegMinSec.dec"], horizontal=True)
geo_format = Label(value= "Lat : " +  _lat_label_format(geo_choice.value) + ', Lon : ' + _lon_label_format(geo_choice.value))

lat_input = Text(description = "Lat : ", placeholder =_lat_label_format(geo_choice.value))
lon_input = Text(description = "Lon : ", placeholder =_lon_label_format(geo_choice.value))
go_button = Button(description = "GO !")


def label_format(option):
    geo_format.value = "Lat : " + _lat_label_format(option.new) + ", Lon : " + _lon_label_format(option.new) 
    lat_input.value = ''
    lon_input.value = ''
    lat_input.placeholder = _lat_label_format(option.new)
    lon_input.placeholder = _lon_label_format(option.new)
    
    
geo_choice.observe(label_format, "value")

# input_control = WidgetControl(widget = VBox([geo_choice, geo_format, lat_input, lon_input, go_button]), position="topright")
# m.add_control(input_control)


In [44]:
last_point_location = None

speed_kts = IntRangeSlider(value = [440, 460], min = 150, max = 500, step = 1, 
                 description = "Speed (kts)", orientation='horizontal', readout=True, readout_format="d")
duration_sec = IntRangeSlider(value = [60, 67], min = 0, max = 900, step = 1, 
                 description = "Duration (s)", orientation='horizontal', readout=True, readout_format="d")

c_ms_md = Circle(color = "green", weight = 3, fill = False)
c_ms_Md = Circle(color = "yellow", weight = 3, fill = False)
c_Ms_md = Circle(color = "orange", weight = 3, fill = False)
c_Ms_Md = Circle(color = "red", weight = 3, fill = False)
m.add_layer(LayerGroup(layers=(c_ms_md, c_ms_Md, c_Ms_md, c_Ms_Md)))

miniRange = Text(placeholder="MiniRange (NM)", description='Dist m (NM) :', disabled=True)
maxiRange = Text(placeholder="MaxiRange (NM)", description='Dist M (NM) :', disabled=True)

def rangeProcessing(event = None):
    mini_sp, maxi_sp = speed_kts.value
    mini_dur, maxi_dur = duration_sec.value
    if last_point_location:
        c_ms_md.location = last_point_location
        c_ms_md.radius = int(convert.speed_conv(mini_sp, from_units='kt', to_units='m/s') * mini_dur)
        
        c_ms_Md.location = last_point_location
        c_ms_Md.radius = int(convert.speed_conv(mini_sp, from_units='kt', to_units='m/s') * maxi_dur)
        
        c_Ms_md.location = last_point_location
        c_Ms_md.radius = int(convert.speed_conv(maxi_sp, from_units='kt', to_units='m/s') * mini_dur)
        
        c_Ms_Md.location =last_point_location
        c_Ms_Md.radius = int(convert.speed_conv(maxi_sp, from_units='kt', to_units='m/s') * maxi_dur)
        
        miniRange.value = str(convert.len_conv(c_ms_md.radius, from_units='m', to_units='nm')) 
        maxiRange.value = str(convert.len_conv(c_Ms_Md.radius, from_units='m', to_units='nm'))  
        
speed_kts.observe(rangeProcessing, "value")  
duration_sec.observe(rangeProcessing, "value")  

In [45]:
gs_kts = IntSlider(value = 400, min = 150, max = 500, step = 1, 
                 description = "Speed (kts)", orientation='horizontal', readout=True, readout_format="d")
bank_angle = IntSlider(value = 25, min = 1, max = 40, step = 1, 
                 description = "Bank angle (°)", orientation='horizontal', readout=True, readout_format="d")

turn_circle = Circle(color = "magenta", weight = 2, fill = False)


m.add_layer(turn_circle)

radius_nm = Text(placeholder="Turn radius (NM)", description='Radius (NM) :', disabled=True)

def radius_computing(event = None):
    gs = convert.speed_conv(gs_kts.value, from_units='kt', to_units='m/s')
    ba = math.radians(bank_angle.value)
    
    if last_point_location:
        turn_circle.location = last_point_location
        
        rad = math.pow(gs, 2) / (constant_g * math.tan(ba))
        turn_circle.radius = int(rad)
        
        radius_nm.value = str(convert.len_conv(turn_circle.radius, from_units='m', to_units='nm'))  
        
gs_kts.observe(radius_computing, "value")  
bank_angle.observe(radius_computing, "value")  

In [46]:
tab = Tab()
tab.set_title(0, "Position")
tab.set_title(1, "Translation")
tab.set_title(2, "Turn action")
tab.children = [VBox([geo_choice, geo_format, lat_input, lon_input, go_button]), VBox([speed_kts, duration_sec, miniRange, maxiRange]), VBox([gs_kts, bank_angle, radius_nm])]

m.add_control(WidgetControl(widget = tab, position="topright"))


In [47]:
# "Deg.dec","DegMin.dec","DegMinSec.dec" 
# "+/-DD.ddd" "N/SDDMM.ddd" "N/SDDMMSS.ddd"
# "+/-DDD.ddd" "W/EDDDMM.ddd" "W/EDDDMMSS.ddd"

r0 = Label(value="--- Square size (m) -------")
r0Lat = Text(placeholder="Dist (m)", description='Lat :', disabled=True)
r0Lon = Text(placeholder="Dist (m)", description='Lon :', disabled=True)

r1 = Label(value="--- Deg.dec -----------")
r1Lat = Text(placeholder="+/-DD.ddd", description='Lat :', disabled=True)
r1Lon = Text(placeholder="+/-DDD.ddd", description='Lon :', disabled=True)

r2 = Label(value="--- DegMin.dec --------")
r2Lat = Text(placeholder="N/SDDMM.ddd", description='Lat :', disabled=True)
r2Lon = Text(placeholder="W/EDDDMM.ddd", description='Lon :', disabled=True)

r3 = Label(value="--- DegMinSec.dec -----")
r3Lat = Text(placeholder="N/SDDMMSS.ddd", description='Lat :', disabled=True)
r3Lon = Text(placeholder="W/EDDDMMSS.ddd", description='Lon :', disabled=True)


result_control = WidgetControl(widget=VBox([r0, r0Lat, r0Lon, r1, r1Lat, r1Lon, r2, r2Lat, r2Lon, r3, r3Lat, r3Lon]), position="topright")
m.add_control(result_control)

In [48]:
def format_lat_ddmm(value):
    if value < 0:
        h = "N"
    else:
        h = "S"
        
    d = int(abs(value))
    m = (value-d) * 60.0
    
    return "{}{:0>2d}{:0>2f}".format(h, d, m)

def format_lon_ddmm(value):
    if value < 0:
        h = "W"
    else:
        h = "E"
        
    d = int(abs(value))
    m = (value-d) * 60.0
    
    return "{}{:0>3d}{:0>2f}".format(h, d, m)

def format_lat_ddmmss(value):
    if value < 0:
        h = "N"
    else:
        h = "S"
        
    d = int(abs(value))
    m = (value-d) * 60.0
    mm = int(abs(m))
    s = (m-mm) * 60.0
    
    return "{}{:0>2d}{:0>2d}{:0>2f}".format(h, d, mm, s)

def format_lon_ddmmss(value):
    if value < 0:
        h = "W"
    else:
        h = "E"
        
    d = int(abs(value))
    m = (value-d) * 60.0 
    mm = int(abs(m))
    s = (m-mm) * 60.0
    
    return "{}{:0>3d}{:0>2d}{:0>2f}".format(h, d, mm, s)
    

In [49]:

def check_lat_dd(value):
    p = "^([+-]?)((\d{2}){1}(\.\d+)?)"
    # print(value)
    if re.fullmatch(p, value):
        tok = value.split('.')
        if abs(int(tok[0])) < 90:
            if len(tok) == 2:
                value_f = float(value)
                lower =  abs(float(tok[0])) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                upper = abs(float(tok[0])) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))
                if value_f < 0:
                    lower_f = -1.0 * upper
                    upper_f = -1.0 * lower
                else:
                    lower_f = lower
                    upper_f = upper
                return lower_f, value_f, upper_f
            elif len(tok) == 1:
                value_f = float(value)
                if value_f < 0:
                    lower_f = value_f-1
                    upper_f = value_f+1
                else:
                    lower_f = value_f-1
                    upper_f = value_f+1
                return lower_f, value_f, upper_f
            else:
                print("Number of tokens")
        else:
            print("Not in range")
    else:
        print("Format is not good")
    return None
            
def check_lon_dd(value):
    p = "^([+-]?)((\d{3}){1}(\.\d+)?)"
    # print(re.match(p, value))
    if re.fullmatch(p, value):
        tok = value.split('.')
        if abs(int(tok[0])) < 180:
            if len(tok) == 2:
                value_f = float(value)

                lower =  abs(float(tok[0])) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                upper = abs(float(tok[0])) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))

                if value_f < 0:

                    lower_f = -1.0 * upper
                    upper_f = -1.0 * lower
                else:
                    lower_f = lower
                    upper_f = upper

                return lower_f, value_f, upper_f
            elif len(tok) == 1:
                value_f = float(value)
                if value_f < 0:
                    lower_f = value_f-1
                    upper_f = value_f+1
                else:
                    lower_f = value_f-1
                    upper_f = value_f+1
                return lower_f, value_f, upper_f
    return None

def check_lat_ddmm(value):
    p = "^([NS]{1})((\d{2}){1}(\d{2}){1}(\.\d+)?)"
    # print(value)
    if re.fullmatch(p, value):
        tok = value.split('.')
        h = tok[0][0]
        if h == "N":
            h = 1.0
        elif h == "S":
            h = -1.0
            
        if (abs(int(tok[0][1:3])) < 90) and (abs(int(tok[0][3:5])) < 60):
            if len(tok) == 2:
                deg = tok[0][1:3]
                mn = tok[0][3:5] 
                dec = tok[1]
                
                mn_d = float(mn + '.' + dec) / 60.0
                value_f = h * (float(deg) + mn_d)
                
                mn_lower = float(mn) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                mn_upper = float(mn) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))
                
                lower_f = h * (float(deg) + mn_lower / 60.0)
                upper_f = h * (float(deg) + mn_upper / 60.0)
                
                return lower_f, value_f, upper_f
            
            elif len(tok) == 1:
                deg = tok[0][1:3]
                mn = tok[0][3:5] 
                
                mn_d = float(mn) / 60.0
                value_f = h * (float(deg) + mn_d)
                
                mn_lower = float(mn) - 1.0
                mn_upper = float(mn) + 1.0
                
                lower_f = h * (float(deg) + mn_lower / 60.0)
                upper_f = h * (float(deg) + mn_upper / 60.0)
                
                return lower_f, value_f, upper_f
            else:
                print("Number of tokens")
        else:
            print("Not in range")
    else:
        print("Format is not good")
    return None

def check_lon_ddmm(value):
    p = "^([WE]{1})((\d{3}){1}(\d{2}){1}(\.\d+)?)"
    # print(value)
    if re.fullmatch(p, value):
        tok = value.split('.')
        h = tok[0][0]
        if h == "E":
            h = 1.0
        elif h == "W":
            h = -1.0
            
        if (abs(int(tok[0][1:4])) < 180) and (abs(int(tok[0][4:6])) < 60):
            if len(tok) == 2:
                deg = tok[0][1:4]
                mn = tok[0][4:6] 
                dec = tok[1]
                
                mn_d = float(mn + '.' + dec) / 60.0
                value_f = h * (float(deg) + mn_d)
                
                mn_lower = float(mn) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                mn_upper = float(mn) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))
                
                lower_f = h * (float(deg) + mn_lower / 60.0)
                upper_f = h * (float(deg) + mn_upper / 60.0)
                
                return lower_f, value_f, upper_f
            elif len(tok) == 1:
                deg = tok[0][1:4]
                mn = tok[0][4:6] 
                
                mn_d = float(mn) / 60.0
                value_f = h * (float(deg) + mn_d)
                
                mn_lower = float(mn) - 1.0
                mn_upper = float(mn) + 1.0
                
                lower_f = h * (float(deg) + mn_lower / 60.0)
                upper_f = h * (float(deg) + mn_upper / 60.0)
                return lower_f, value_f, upper_f
            else:
                print("Number of tokens")
        else:
            print("Not in range")
    else:
        print("Format is not good")
    return None

def go_action(event):
    global last_point_location
    # print('--------------------------------------')
    # # "Deg.dec","DegMin.dec","DegMinSec.dec"
    # print(geo_choice.value)
    if geo_choice.value == "Deg.dec" :
        lat_l = check_lat_dd(lat_input.value)
        lon_l = check_lon_dd(lon_input.value)
    elif geo_choice.value == "DegMin.dec" :
        lat_l = check_lat_ddmm(lat_input.value)
        lon_l = check_lon_ddmm(lon_input.value)
    elif geo_choice.value == "DegMinSec.dec" :
        pass
      
    # print(lat_l)
    # print(lon_l)
    if lat_l and lon_l:
        last_point_location = (lat_l[1], lon_l[1])
        _, _, dLat = geo.inv(lat_l[0], lon_l[0], lat_l[2], lon_l[0])
        _, _, dLon = geo.inv(lat_l[0], lon_l[0], lat_l[0], lon_l[2])
        r0Lat.value = str(dLat) 
        r0Lon.value = str(dLon) 
        r1Lat.value = str(lat_l[1]) 
        r1Lon.value = str(lon_l[1]) 
        r2Lat.value = format_lat_ddmm(lat_l[1])
        r2Lon.value = format_lon_ddmm(lon_l[1])
        r3Lat.value = format_lat_ddmmss(lat_l[1])
        r3Lon.value = format_lon_ddmmss(lon_l[1])
        
        marker = Marker(location=(lat_l[1], lon_l[1]), draggable=False)
        m.add_layer(marker)
        rectangle = Rectangle(bounds=((lat_l[0], lon_l[0]), (lat_l[2], lon_l[2])), weight = 1)
        m.add_layer(rectangle)
    else:
        last_point_location = None
        
    rangeProcessing()
    radius_computing()

go_button.on_click(go_action)

In [50]:
m


Map(center=[46.5, 6.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…